In [343]:
import numpy as np
import sympy as sp
import pickle
import os

In [344]:
# Define the output directory
output_dir = "./thr_output"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [374]:
# Symbolically compute the expected value vector, µ_n = [S_n; R_n].

# All theory is derived in 'Revisiting the Luria_Delbrük experiment:
#                           The case of reversible non-genetic resistance'

# p, q: real numbers in [0, 1]
p = sp.Symbol('p', real=True, nonnegative=True) # probability of becoming resistant from sensitive
q = sp.Symbol('q', real=True, nonnegative=True) # probability of becoming sensitive from resistant

# n: integer
n = sp.Symbol('n', integer=True, nonnegative=True) # number of generations

# dphi(i)/ds is the expected number of sensitive offspring produced by a parent of the ith type
# dphi(i)/dt is the expected number of resistant offspring produced by a parent of the ith type
dphi1_ds = 2 * (1-p)
dphi1_dt = 2 * p
dphi2_ds = 2 * q
dphi2_dt = 2 * (1-q)

# matrix of expectations
M = sp.Matrix([
    [dphi1_ds, dphi1_dt],
    [dphi2_ds, dphi2_dt]
])

# compute the eigenvalue, eigenvector decomposition of M^T
# s.t. M^T = X * D * X^-1
X, D = (M.T).diagonalize()
X_inv = X.inv()


# initial conditions for mean, µ0_1, µ0_2: real numbers
mu_0_S = sp.Symbol('\mu_0^{(S)}', real=True, nonnegative=True)
mu_0_R = sp.Symbol('\mu_0^{(R)}', real=True, nonnegative=True)
mu_0 = sp.Matrix([[mu_0_S], [mu_0_R]])

# the formula for expected value is µ_n = (M^T)^n * u_0
# which can be simplified to µ_n = X * D^n * X^-1 * u_0
# for easy computation.
mu_n = sp.simplify(X * D**n * X_inv * mu_0)

# convert the formula into a numerical expression
mu_n_num = sp.lambdify([p, q, n, mu_0_S, mu_0_R], mu_n)

# export the symbolic expression
with open(os.path.join(output_dir, 'mu_n.pkl'), 'wb') as file:
    pickle.dump(mu_n, file)
    
# to open the expression, use:
# ```
# import sympy as sp
# import pickle
# with open('mu_n.pkl', 'rb') as file:
#     mu_n = pickle.load(file)

In [375]:
mu_n

Matrix([
[(\mu_0^{(R)}*q*(2**n - (-2*p - 2*q + 2)**n) + \mu_0^{(S)}*(2**n*q + p*(-2*p - 2*q + 2)**n))/(p + q)],
[(\mu_0^{(R)}*(2**n*p + q*(-2*p - 2*q + 2)**n) + \mu_0^{(S)}*p*(2**n - (-2*p - 2*q + 2)**n))/(p + q)]])

In [376]:
# Symbolically compute the variance vector, V_n = [Var(S_n); Cov(S_n, R_n); Var(R_n)].
# All theory is derived in 'Revisiting the Luria_Delbrük experiment:
#                           The case of reversible non-genetic resistance'

# k: integer
k = sp.Symbol('k', integer=True, nonnegative=True) # dummy variable for n in a summation

# compute the variances/covariances for the number of offpsring produced by a single parent of type i
# store these in the matrix Sigma
sigmaS_1  =  4*p * (1-p)
sigmaSR_1 = -4*p * (1-p)
sigmaR_1  =  4*p * (1-p)

sigmaS_2  =  4*q * (1-q)  
sigmaSR_2 = -4*q * (1-q)  
sigmaR_2  =  4*q * (1-q)

Sigma = sp.Matrix([
    [sigmaS_1,  sigmaS_2],
    [sigmaSR_1, sigmaSR_2],
    [sigmaR_1,  sigmaR_2]
])

# L matrix as defined in the theory derivations
L = sp.Matrix([
    [M[0,0]**2,               2*M[0,0]*M[1,0],               M[1,0]**2],
    [M[0,0]*M[0,1],     M[0,0]*M[1,1]+M[0,1]*M[1,0],     M[1,0]*M[1,1]],
    [M[0,1]**2,               2*M[0,1]*M[1,1],               M[1,1]**2]
])

# substitue the dummy variable k into the symbolic expression for mu_n
mu_k = mu_n.subs({n: k})

# diagonalize L to make the matrix computations (raising to power) easier
P, W  = L.diagonalize()
P_inv = P.inv()
W_inv = W.inv()

# initial conditions for variance, v0_1, v0_2, v0_3: real numbers
v_0_S  = sp.Symbol('v_0^{(S)}', real=True, nonnegative=True)
v_0_SR = sp.Symbol('v_0^{(SR)}', real=True, nonnegative=True)
v_0_R  = sp.Symbol('v_0^{(R)}', real=True, nonnegative=True)
v_0    = sp.Matrix([[v_0_S], [v_0_SR], [v_0_R]])

# v_n = P * W^n(P_inv*v_0 + W_inv*∑_{k=0}^{n-1}(W^{-k} * P_inv*Sigma*X * D^k) * X_inv*µ_0)
# we will define the summation term as sum_term and compute this first

# Let's define K(mu) = W^{-k} * P_inv * Sigma * X * D^k to avoid the repeated computation
K = W**(-k) * P_inv * Sigma * X * D**k

# now compute the sum term 
# since K is a 3 x 2 matrix, compute each element of the sum separately using applyfunc
# sp.summation only works with scalar expressions
sum_term = K.applyfunc(lambda term: sp.summation(term, (k, 0, n-1)))

# use the sum_term to compute v_n using the formula for v_n shown above
v_n = P * W**n * (P_inv * v_0 + W_inv * sum_term * X_inv * mu_0)
v_n = sp.simplify(v_n)

# convert the formula into a numerical expression
v_n_num = sp.lambdify([p, q, n, mu_0_S, mu_0_R, v_0_S, v_0_SR, v_0_R], v_n)

# export the symbolic expression
with open(os.path.join(output_dir, 'v_n.pkl'), 'wb') as file:
    pickle.dump(v_n, file)
    
# to open the expression, use:
# ```
# import sympy as sp
# import pickle
# with open('v_n.pkl', 'rb') as file:
#     v_n = pickle.load(file)

In [377]:
v_n

Matrix([
[Piecewise(((4**n*q**2*(p + q)**3*(v_0^{(R)} + v_0^{(S)} + 2*v_0^{(SR)})*(p**2 + 2*p*q - 2*p + q**2 - 2*q + 1) + 2*q*(4*(-p - q + 1))**n*(p + q)**3*(p*v_0^{(S)} - q*v_0^{(R)} + v_0^{(SR)}*(p - q))*(p**2 + 2*p*q - 2*p + q**2 - 2*q + 1) + (4*(p**2 + 2*p*q - 2*p + q**2 - 2*q + 1))**n*(n*(\mu_0^{(R)}*q*(p**4 + 2*p**3*q - p**3 - p**2*q - 2*p*q**3 + p*q**2 - p*(p**3 + 3*p**2*q - 2*p**2 + 3*p*q**2 - 4*p*q + q**3 - 2*q**2) - q**4 + q**3) - \mu_0^{(S)}*p*(p**4 + 2*p**3*q - p**3 - p**2*q - 2*p*q**3 + p*q**2 - q**4 + q**3 + q*(p**3 + 3*p**2*q - 2*p**2 + 3*p*q**2 - 4*p*q + q**3 - 2*q**2)))*(p**2 + 2*p*q + q**2) + (p + q)**3*(p**2*v_0^{(S)} - 2*p*q*v_0^{(SR)} + q**2*v_0^{(R)})*(p**2 + 2*p*q - 2*p + q**2 - 2*q + 1)))/((p + q)**3*(p**2 + 2*p*q + q**2)*(p**2 + 2*p*q - 2*p + q**2 - 2*q + 1)), Eq((p + q - 1)**(-2), 2) & Eq(1/(p + q - 1), -2)), ((1/(2*(p + q - 1)**2))**n*(q*(2*(p + q - 1)**2)**n*(p + q)**3*(4**n*q*(v_0^{(R)} + v_0^{(S)} + 2*v_0^{(SR)}) + 2*(4*(-p - q + 1))**n*(p*v_0^{(S)} - q*v_

In [382]:
# ON PEN AND PAPER, I SIMPLIFIED THE SYMBOLIC FUNCTIONS OUTPUTTED BY SYMPY AND REWROTE THEM HERE
# I VALIDATED THESE FUNCTIONS AGAINST SYMPY

def pi(p_val, q_val):
    '''
    pi exports the steady state levels for the mean of the two populations.
    '''
    pi_S = q_val / (p_val + q_val)
    pi_R = p_val / (p_val + q_val)
    return pi_S, pi_R

def delta(p_val, q_val):
    '''
    delta exports the relaxation parameter of the system which describes the time
    it takes for the system to reach equilibrium.
    '''
    return 1 - p_val - q_val

def rec_mean(p_val, q_val, n_gen, mu_0):
    '''
    This function computes the mean using the recurrent formulation: µ_n = M.T x µ_{n-1}
    '''

    if n_gen == 0:
        return mu_0
    
    M_T = np.array([
        [2*(1-p_val), 2*p_val],
        [2*q_val, 2*(1-q_val)]
    ]).T
    
    return M_T @ rec_mean(p_val, q_val, n_gen-1, mu_0)

def manual_mu_n(p_val, q_val, n_gen, mu_0):
    '''
    This function implements a more readable version of the mu_n symbolic function
    from sympy. Note that this implementation suffers from numerical errors that the
    sympy version does not.
    '''
    try:
        pi_S, pi_R = pi(p_val, q_val)
        d          = delta(p_val, q_val)

        mu_matrix = np.array([
            [pi_S*(1 + p_val/q_val * d**n_gen),         pi_S*(1 - d**n_gen)],
            [        pi_R*(1 - d**n_gen),          pi_R*(1 + q_val/p_val * d**n_gen)]
        ])

        expected_value = (2**(n_gen) * mu_matrix) @ mu_0
        return expected_value
    except:
        return np.array([np.nan, np.nan])
    
def rec_var(p_val, q_val, n_gen, mu_0, v_0):
    '''
    This function computes the variance using the recurrent formulation: v_n = L x v_{n-1} + Sigma x u_{n-1}
    '''

    if n_gen == 0:
        return v_0
    
    L = np.array([
        [4*(1-p_val)**2,             8*q_val*(1-p_val),                 4*q_val**2],
        [4*p_val*(1-p_val), 4*(1-p_val)*(1-q_val)+4*p_val*q_val, 4*q_val*(1-q_val)],
        [4*p_val**2,                 8*p_val*(1-q_val),             4*(1-q_val)**2]
    ])
    
    Sigma = np.array([
        [ 4*p_val*(1-p_val),  4*q_val*(1-q_val)],
        [-4*p_val*(1-p_val), -4*q_val*(1-q_val)],
        [ 4*p_val*(1-p_val),  4*q_val*(1-q_val)]
    ])

    return L @ rec_var(p_val, q_val, n_gen-1, mu_0, v_0) + Sigma @ rec_mean(p_val, q_val, n_gen-1, mu_0)

def manual_v_n(p_val, q_val, n_gen, mu_0, v_0, epsilon=1e-15):
    '''
    This function implements a more readable version of the v_n symbolic function
    from sympy. Note that this implementation suffers from numerical errors that the
    sympy version does not.
    '''
    try:
        pi_S, pi_R = pi(p_val, q_val)
        d          = delta(p_val, q_val)

        first_term  = 2**(2*n_gen) * np.outer(np.array([pi_S**2, pi_S*pi_R, pi_R**2]), np.array([1, 2, 1])) @ v_0

        second_term_matrix = np.array([
            [    2*pi_S*pi_R,     -2*(pi_S-pi_R)*pi_S,     -2*pi_S**2],
            [-(pi_S-pi_R)*pi_R,     (pi_S-pi_R)**2,     (pi_S-pi_R)*pi_S],
            [   -2*pi_R**2,        2*(pi_S-pi_R)*pi_R,      2*pi_S*pi_R]
        ])
        second_term = 2**(2*n_gen) * d**n_gen * second_term_matrix @ v_0

        third_term  = (2*d)**(2*n_gen) * np.array([1, -1, 1])

        if abs(p_val + q_val - (1/np.sqrt(2)) - 1) < epsilon:
            geom_term_sqrt = n_gen
        else:
            geom_term_sqrt = (1 - (2*d**2)**(-n_gen)) / (1 - (2*d**2)**(-1))

        if abs(p_val + q_val - (1/2)) < epsilon:
            geom_term_half = n_gen
        else:
            geom_term_half = (1 - (2*d)**(-n_gen)) / (1 - (2*d)**(-1))

        mu_S_term_1 = (q_val * (d+1) * geom_term_sqrt)  / (d**2)
        mu_S_term_2 = ((q_val-p_val) * geom_term_half) / d

        mu_R_term_1 = (p_val * (d+1) * geom_term_sqrt)  / (d**2)
        mu_R_term_2 = ((q_val - p_val) * geom_term_half) / d

        mu_terms_coeffs = np.array([pi_R*(mu_S_term_1 - mu_S_term_2), pi_S*(mu_R_term_1 + mu_R_term_2)])
        mu_terms        = (mu_terms_coeffs @ mu_0) * np.array([1, 1, 1])

        var_terms = (np.outer(np.array([pi_R**2, -2*pi_S*pi_R, pi_S**2]), np.array([1, 1, 1]))).T @ v_0

        variance = first_term + second_term + third_term * (mu_terms + var_terms)

        return variance
    except:
        return np.array([np.nan, np.nan, np.nan])